#### Imports

In [1]:
from environments import build_blue_agent, build_red_agent, sample

import ray
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
import logging
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
logger = logging.getLogger(__name__)


/home/garrett/miniconda3/envs/cyborg/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:513: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/home/garrett/miniconda3/envs/cyborg/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
/home/garrett/miniconda3/envs/cyborg/lib/python3.8/site-packages/tensorflow/python/fra

#### Verify GPU

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### Load Agents

In [3]:
# Start Ray
ray.init(num_gpus=1, ignore_reinit_error=True, log_to_driver=False)

comp_blue = build_blue_agent(workers=1)
path_file = open(f"./policies/competitive_blue_policy", "r")
blue_restore_path = path_file.read()
path_file.close()
comp_blue.restore(blue_restore_path)

comp_red = build_red_agent(workers=1)
path_file = open(f"./policies/competitive_red_policy", "r")
red_restore_path = path_file.read()
path_file.close()
comp_red.restore(red_restore_path)

ded_blue = build_blue_agent(dedicated=True, workers=1)
path_file = open(f"./policies/dedicated_blue_policy", "r")
blue_restore_path = path_file.read()
path_file.close()
ded_blue.restore(blue_restore_path)

ded_red = build_red_agent(dedicated=True, workers=1)
path_file = open(f"./policies/dedicated_red_policy", "r")
red_restore_path = path_file.read()
path_file.close()
ded_red.restore(red_restore_path)

sample_size = 1000

./policies/blue_competitive_pool/competitive_blue_0/checkpoint_000000
./policies/red_competitive_pool/competitive_red_0/checkpoint_000000
./policies/blue_dedicated_pool/dedicated_blue_0/checkpoint_000000
./policies/red_dedicated_pool/dedicated_red_0/checkpoint_000000


#### Score Matrix

In [4]:
print(f"Competitive Blue vs Competitive Red: {sample(comp_red, comp_blue, games=sample_size):0.2f}")
print(f"Competitive Blue vs Dedicated Red: {sample(ded_red, comp_blue, games=sample_size):0.2f}")
print(f"Competitive Blue vs Random Red: {sample(comp_red, comp_blue, games=sample_size, random_red=True):0.2f}")
print(f"Dedicated Blue vs Competitive Red: {sample(comp_red, ded_blue, games=sample_size):0.2f}")
print(f"Dedicated Blue vs Dedicated Red: {sample(ded_red, ded_blue, games=sample_size):0.2f}")
print(f"Dedicated Blue vs Random Red: {sample(comp_red, ded_blue, games=sample_size, random_red=True):0.2f}")
print(f"Random Blue vs Competitive Red: {sample(comp_red, comp_blue, games=sample_size, random_blue=True):0.2f}")
print(f"Random Blue vs Dedicated Red: {sample(ded_red, comp_blue, games=sample_size, random_blue=True):0.2f}")
print(f"Random Blue vs Random Red: {sample(comp_red, comp_blue, games=sample_size, random_red=True, random_blue=True):0.2f}")

Competitive Blue vs Competitive Red: 4.82
Competitive Blue vs Dedicated Red: 5.47
Competitive Blue vs Random Red: 0.03
Dedicated Blue vs Competitive Red: 4.29
Dedicated Blue vs Dedicated Red: 4.98
Dedicated Blue vs Random Red: 0.03
Random Blue vs Competitive Red: 12.85
Random Blue vs Dedicated Red: 12.03
Random Blue vs Random Red: 4.07
